# Домашнее задание 1 #

*Программа работает с файлом "hw_3_twitter.json", расположенным в одной папке с программой. Файл считывается и построчно превращается в словари Python.*

In [1]:
import json
import collections
import string
from string import punctuation
import re
from pprint import pprint

twitter = []
with open("hw_3_twitter.json", encoding="utf-8") as f:
    for line in f:
        twitter.append(json.loads(line))

## Пункты 1-7 ##

### Пункт 1 ###
Поскольку каждая строка - это один твит, выясним, сколько твитов в наборе, посчитав длину получившегося в предыдущем действии списка (twitter).

In [2]:
twit_amount = len(twitter)
print("Количество твитов в наборе составляет", twit_amount)

Количество твитов в наборе составляет 2556


### Пункт 2 ###
Найдем процент удаленных твитов, посчитав количество твитов с пометкой "delete" и разделив его на общее количество твитов (значение округлено до сотых).

In [3]:
del_twit_amount = 0
for tweet in twitter:
    if "delete" in tweet:
        del_twit_amount += 1
if twit_amount:  # мы точно не знаем, что общее число твитов больше 0, поэтому защитимся от деления на 0
    del_percentage = round((del_twit_amount / twit_amount) * 100, 2)
    print("Процент удаленных твитов составляет", del_percentage, "%")

Процент удаленных твитов составляет 14.16 %


### Пункт 3 ###
Выясним, какие языки наиболее популярны. Для этого сначала составим список из всех языков, встретившихся в твитах (lang_list), при этом исключив из рассмотрения удаленные твиты, т.к. они не содержат информации о языке. Далее на основе полученного списка составим частотный словарь языков (freq_lang), отсортируем словарь по значениям в обратном порядке и распечатаем первые 10 пар "ключ: значение".

In [4]:
lang_list = []
for tweet in twitter:
    if "delete" not in tweet:
        lang = tweet["lang"]
        lang_list.append(lang)

freq_lang = collections.Counter(lang_list)

print("Топ-10 самых популярных языков:")
for number, key in enumerate(sorted(freq_lang, key=freq_lang.get, reverse=True)):
    if number < 10:
        print(key, "-", freq_lang[key], "твитов")

Топ-10 самых популярных языков:
en - 719 твитов
ja - 438 твитов
es - 173 твитов
ko - 149 твитов
th - 123 твитов
ar - 119 твитов
und - 117 твитов
in - 71 твитов
pt - 69 твитов
fr - 35 твитов


### Пункт 4 ###
Узнаем, есть ли твиты от одного и того же пользователя (**включая удаленные твиты**). Будем по одному добавлять id пользователей в список user_ids (для не-удаленных твитов id содержится внутри tweet["user"]["id"], для удаленных - в поле tweet["delete"]["status"]["user_id"]). Таким образом, в списке user_ids каждое имя пользователя будет встречаться столько раз, сколько твитов написал этот пользователь.
Далее создадим частотный словарь с именами пользователей, добавим тех, кто написал более одного твита, в список repeated_users и выведем длину этого списка.

In [5]:
user_ids = []
for tweet in twitter:
    if "user" in tweet:
        user_id = tweet["user"]["id"]
    if "delete" in tweet:
        user_id = tweet["delete"]["status"]["user_id"]
    user_ids.append(user_id)

users = collections.Counter(user_ids)
repeated_users = []
for user in users:
    if users[user] > 1:
        repeated_users.append(user)
print("Количество пользователей, написавших больше одного твита, составляет", len(repeated_users))

Количество пользователей, написавших больше одного твита, составляет 66


### Пункт 4 (вариант 2) ###
На всякий случай - вариант **без учета удаленных твитов**. Алгоритм работы кода в целом тот же, что и в первом случае.

In [6]:
user_ids = []
for tweet in twitter:
    if "delete" not in tweet:
        user_id = tweet["user"]["id"]
        user_ids.append(user_id)

users = collections.Counter(user_ids)
repeated_users = []
for user in users:
    if users[user] > 1:
        repeated_users.append(user)
print("Количество пользователей, написавших больше одного твита, составляет", len(repeated_users))

Количество пользователей, написавших больше одного твита, составляет 25


### Пункт 5 ###
Выясним, какие хэштеги наиболее популярны (будем работать только с не-удаленными твитами, т.к. у удаленных тегов нет). Соберем все хэштеги, хранящиеся внутри ["entities"]["hashtags"], в общий список hashtags, и удалим из этого списка пустые элементы с помощью filter. Поскольку каждым элементом hashtags является не отдельный хэштег, а список хэштегов (+ дополнительная информация о них) из какого-либо твита, с помощью вложенного цикла вытащим собственно теги из ["entities"]["hashtags"]["text"] и сложим их в список cleared_hashtags. Далее создадим частотный словарь хэштегов, отсортируем его по убыванию значений и запишем первые 20 ключей в список top20_list, который затем "красиво" выведем.

In [7]:
hashtags = []
for tweet in twitter:
    if "delete" not in tweet:
        hashtag_list = tweet["entities"]["hashtags"]
        hashtags.append(hashtag_list)

hashtags = list(filter(None, hashtags))

cleared_hashtags = []
for element in hashtags:
    for tags in element:
        tag = tags["text"]
        cleared_hashtags.append(tag)

print("Топ-20 хэштегов:")
top_hashtags = collections.Counter(cleared_hashtags)
for number, hashtag in enumerate(sorted(top_hashtags, key=top_hashtags.get, reverse=True)):
    if number < 20:
        print(hashtag, "-", top_hashtags[hashtag], "употреблений")

Топ-20 хэштегов:
BTS - 17 употреблений
방탄소년단 - 13 употреблений
AMAs - 11 употреблений
人気投票ガチャ - 8 употреблений
태형 - 7 употреблений
뷔 - 6 употреблений
BTSinChicago - 5 употреблений
BTSLoveYourselfTour - 5 употреблений
오늘의방탄 - 5 употреблений
PledgeForSwachhBharat - 5 употреблений
MPN - 5 употреблений
PCAs - 4 употреблений
V - 4 употреблений
시카고1회차공연 - 4 употреблений
เป๊กผลิตโชค - 4 употреблений
JIMIN - 4 употреблений
running - 3 употреблений
NCT - 3 употреблений
지민 - 3 употреблений
WajahmuPlastik - 3 употреблений


### Пункт 6 ###
Составим частотный словарь всех слов в оригинальных твитах на английском языке. Сначала соберем все тексты подходящих (англоязычных, не-ретвитнутых и не-удаленных) твитов в список en_tweet_texts. Чтобы получить эти тексты, нужно использовать:

1) поле "extended_tweet"["full_text"], если оно есть (а есть оно не во всех твитах);

2) поле "text" в остальных случаях.

Далее пройдемся по каждому тексту в цикле и с помощью split разделим тексты на отдельные слова/числа/ссылки/эмодзи и т.д., после чего получившиеся единицы запишем в общий список full_tokens.
Теперь обработаем получившиеся "слова". Приведем их к нижнему регистру с помощью lower, избавимся от эмодзи с помощью encode и decode, проверим, что слово не является ссылкой, хэштегом или обращением к пользователю (не начинается на http, # или @), после чего удалим знаки препинания, убедимся, что слово действительно является словом (а не, например, числом) и добавим его в общий список words. На основе полученного списка создадим частотный словарь freq_dict, отсортируем его по значениям в обратном порядке и выведем первые 10 пар "ключ: значение".

In [8]:
en_tweet_texts = []
for tweet in twitter:
    if "delete" not in tweet and "retweeted_status" not in tweet and "quoted_status" not in tweet and tweet["lang"] == "en":
        if "extended_tweet" in tweet:
            en_tweet_texts.append(tweet["extended_tweet"]["full_text"])
        else:
            en_tweet_texts.append(tweet["text"])

full_tokens = []
for tweet in en_tweet_texts:
    tokens = tweet.split()
    full_tokens.extend(tokens)   

words = []
for token in full_tokens:
    word = ((token.lower()).encode('ascii', 'ignore')).decode('ascii')
    if not (word.startswith("http") or word.startswith("#") or word.startswith("@")):
        word = word.translate(word.maketrans("", "", string.punctuation))
        if word.isalpha():
            words.append(word)

print("Топ-20 наиболее частотных слов:")
freq_dict = collections.Counter(words)
for number, word in enumerate(sorted(freq_dict, key=freq_dict.get, reverse=True)):
    if number < 20:
        print(word, "-", freq_dict[word], "вхождений")

Топ-20 наиболее частотных слов:
the - 134 вхождений
to - 96 вхождений
a - 74 вхождений
and - 69 вхождений
i - 68 вхождений
you - 54 вхождений
of - 51 вхождений
is - 50 вхождений
it - 49 вхождений
for - 44 вхождений
in - 43 вхождений
that - 35 вхождений
me - 31 вхождений
on - 30 вхождений
my - 27 вхождений
be - 25 вхождений
its - 24 вхождений
this - 22 вхождений
are - 22 вхождений
your - 21 вхождений


### Пункт 7 ###
Выясним, у кого из пользователей наибольшее число подписчиков. Будем работать только с не-удаленными твитами, поскольку в удаленных эта информация отсутствует.
Из каждого твита вытащим имя пользователя, хранящееся в поле tweet["user"]["screen_name"], и количество подписчиков из поля tweet["user"]["followers_count"]. Затем запишем полученную информацию в словарь followers_leaders в формате "имя пользователя: количество подписчиков", каждый раз проверяя, что данных этого пользователя еще нет в словаре.
Отсортировав полученный словарь по значениям в обратном порядке, распечатаем первые 10 пар "ключ: значение", добавляя к данным подписи для "красоты".

In [9]:
followers_leaders = {}
for tweet in twitter:
    if "delete" not in tweet:
        username = tweet["user"]["screen_name"]
        follower_count = tweet["user"]["followers_count"]
        if username not in followers_leaders:
            followers_leaders[username] = follower_count
        
print("Топ-10 пользователей с наибольшим количеством подписчиков:")
for number, key in enumerate(sorted(followers_leaders, key=followers_leaders.get, reverse=True)):
    if number < 10:
        string = str(number + 1) + " место - пользователь " + key + ": " + str(followers_leaders[key]) + " подписчиков"
        print(string)

Топ-10 пользователей с наибольшим количеством подписчиков:
1 место - пользователь FIL0S0FIA: 2521403 подписчиков
2 место - пользователь FitnessMagazine: 1491309 подписчиков
3 место - пользователь malaysiakini: 1206759 подписчиков
4 место - пользователь NYTScience: 1137374 подписчиков
5 место - пользователь GramaticaReal: 625463 подписчиков
6 место - пользователь tgrthabertv: 392472 подписчиков
7 место - пользователь TheSunFootball: 383698 подписчиков
8 место - пользователь Melbourne: 374222 подписчиков
9 место - пользователь Roznama_Express: 318189 подписчиков
10 место - пользователь burger_boogie: 311319 подписчиков


## Пункт 8 ##
Выясним, каковы самые распространенные приложения для Твиттера. Вновь будем работать только с не-удаленными твитами.
Ситуация различается для оригинальных твитов и ретвитов: у оригинального твита только один источник, а у ретвита два - источник самого твита и источник ретвита. **Учтем все случаи**.
Для оригинальных твитов извлечем источник из поля "source" при помощи регулярного выражения и запишем его в список sources. Для ретвитов найдем источник оригинального твита (поле "retweeted_status"["source"]) и источник ретвита (поле "source") и запишем их в тот же список.
Создадим частотный словарь источников твитов (sources_freq), отсортируем его по значениям в обратном порядке и распечатаем первые 10 элементов.

In [10]:
sources = []
for tweet in twitter:
    if "delete" not in tweet:
        if "retweeted_status" not in tweet:
            source = (re.search('<a href="(.*?)>(.*?)</a>', tweet["source"])).group(2)
        else:
            source = (re.search('<a href="(.*?)>(.*?)</a>', tweet["retweeted_status"]["source"])).group(2)
            retweet_source = (re.search('<a href="(.*?)>(.*?)</a>', tweet["source"])).group(2)
            sources.append(retweet_source)
        sources.append(source)

sources_freq = collections.Counter(sources)
print("Топ-10 источников твита:")
for number, source in enumerate(sorted(sources_freq, key=sources_freq.get, reverse=True)):
    if number < 10:
        print(source, "-", sources_freq[source], "твитов")

Топ-10 источников твита:
Twitter for iPhone - 1281 твитов
Twitter for Android - 964 твитов
Twitter Web Client - 303 твитов
twittbot.net - 123 твитов
Twitter Lite - 65 твитов
TweetDeck - 41 твитов
Twitter for iPad - 38 твитов
IFTTT - 20 твитов
Facebook - 19 твитов
Hootsuite Inc. - 18 твитов


## Пункт 8 (вариант 2) ##
На всякий случай - вариант, учитывающий только источники твитов "верхнего слоя" (т.е. не учитывающий источники ретвитнутых твитов, а учитывающий только источник самого ретвита).

In [11]:
sources = []
for tweet in twitter:
    if "delete" not in tweet:
        source = (re.search('<a href="(.*?)>(.*?)</a>', tweet["source"])).group(2)
        sources.append(source)

sources_freq = collections.Counter(sources)
print("Топ-10 источников твита:")
for number, source in enumerate(sorted(sources_freq, key=sources_freq.get, reverse=True)):
    if number < 10:
        print(source, "-", sources_freq[source], "твитов")

Топ-10 источников твита:
Twitter for iPhone - 800 твитов
Twitter for Android - 695 твитов
Twitter Web Client - 140 твитов
twittbot.net - 122 твитов
Twitter Lite - 51 твитов
Twitter for iPad - 28 твитов
TweetDeck - 23 твитов
Facebook - 17 твитов
IFTTT - 14 твитов
تطبيق قرآني - 10 твитов
